In [95]:
from typing import Any, Dict, Tuple, Callable, Optional

import numpy as np
import random

from check_submission import check_submission
from game_mechanics import GoEnv, choose_move_randomly, load_pkl, play_go, save_pkl

from torch import nn
import torch
import torch.nn.functional as F

TEAM_NAME = "Team Bulgar Stronk"  # <---- Enter your team name here!
assert TEAM_NAME != "Team Name", "Please change your TEAM_NAME!"

In [89]:
def choose_move(observation: np.ndarray, legal_moves: np.ndarray, neural_network: nn.Module) -> int:
    """Called during competitive play. It acts greedily given current state of the board and value
    function dictionary. It returns a single move to play.

    Args:
        state:

    Returns:
    """
    illegal_moves = [_ for _ in range(82) if _ not in legal_moves]
    # todo: 
    # pass board state to neural network
    # get results
    obs = torch.as_tensor(observation, dtype=torch.float32)
#     print("============================")
    probs, value = my_network(obs, illegal_moves)
    probs = probs.cpu().detach().numpy()
#     cumulatives = probs.cumsum(0)
#     idx = torch.searchsorted(cumulatives, torch.rand(1))
#     A[idx], idx
    np.random.choice(range(0, 82), p=probs)
#     print(probs)
#     print("============================")
    return random.choice(legal_moves)

def choose_move_no_network(observation: np.ndarray, legal_moves: np.ndarray) -> int:
    """The arguments in play_game() require functions that only take the state as input.

    This converts choose_move() to that format.
    """
    return choose_move(observation, legal_moves, my_network)


def choose_move_human(observation: np.ndarray, legal_moves:np.ndarray, neural_network: nn.Module) -> int:
    print(observation)
    i, j = [int(_) for _ in input().split(" ")]

    return (i-1)*9 + j-1


def choose_move_no_network_human(observation: np.ndarray, legal_moves: np.ndarray) -> int:
    """The arguments in play_game() require functions that only take the state as input.

    This converts choose_move() to that format.
    """
    return choose_move_human(observation, legal_moves, my_network)

In [91]:
# use this to manually create a go env and run a few moves

your_choose_move = choose_move_no_network_human
opponent_choose_move = choose_move_no_network
game_speed_multiplier=1
render=False
verbose=True


env = GoEnv(
    opponent_choose_move,
    verbose=verbose,
    render=render,
    game_speed_multiplier=game_speed_multiplier,
)

observation, reward, done, info = env.reset()
done = False
for i in range(2): # while not done:
    action = your_choose_move(observation, info["legal_moves"])
    observation, reward, done, info = env.step(action)
    print(observation)
print("end")

Resetting Game.
You are playing with the black tiles.
Black plays first


[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
3 3


C:\Users\a816412\AppData\Local\Temp/ipykernel_19516/3011500494.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x4a = F.softmax(x4a)


black places counter at coordinate: (2, 2)
white places counter at coordinate: (5, 7)
[[ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]
[[ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]


KeyboardInterrupt: Interrupted by user

In [68]:
class alpha_go_zero(nn.Module):
    def __init__(self):
        super().__init__()

        # This represents the shared layer(s) before the different heads

        self.flatten = nn.Flatten(0, -1)
        
        self.layer1 = nn.Linear(81, 600)
        self.layer2 = nn.Linear(600, 600)
        
        self.head1 = nn.Linear(600, 82)
        self.head2 = nn.Linear(600, 1)        

    def forward(self, x, illegal_moves):

        # Run the shared layer(s)
        x1 = self.flatten(x)
        x1 = self.layer1(x1)
        x1 = F.elu(x1)
        
        x2 = self.layer2(x1)
        x2 = F.elu(x2)
        
        x3 = self.layer2(x2)
        x3 = F.elu(x3)
        
        
        # Run the different heads with the output of the shared layers as input
        # stochastic predictions
        x4a = self.head1(x3)
        x4a[illegal_moves] = -torch.inf
        x4a = F.softmax(x4a)
        
        #value function
        x4b = self.head2(x3)
        
        return x4a, x4b

my_network = alpha_go_zero()

In [ ]:
def train() -> nn.Module:
    """
    TODO: Write this function to train your algorithm.

    Returns:
    """
    
    # za 1000 igri
    # zapo4vame ot na4alo do krai s probs
    # pravim 1 greshka - loss na hodovete sys y
    # pravim 2 greshka - na evaluatora sys y
    # zapisvame episodic memory
    
    
    your_choose_move = choose_move_no_network_human
    opponent_choose_move = choose_move_no_network
    game_speed_multiplier=1
    render=False
    verbose=True


    env = GoEnv(
        opponent_choose_move,
        verbose=verbose,
        render=render,
        game_speed_multiplier=game_speed_multiplier,
    )

    observation, reward, done, info = env.reset()
    done = False
    for i in range(2): # while not done:
        action = your_choose_move(observation, info["legal_moves"])
        observation, reward, done, info = env.step(action)
        print(observation)
    print("end")

In [4]:
## Example workflow, feel free to edit this! ###
# file = train()
# save_pkl(file, TEAM_NAME)

# my_network = load_pkl(TEAM_NAME)


# Code below plays a single game against a random
#  opponent, think about how you might want to adapt this to
#  test the performance of your algorithm.


# check_submission(
#     TEAM_NAME, choose_move_no_network
# )  # <---- Make sure I pass! Or your solution will not work in the tournament!!

play_go(
    your_choose_move=choose_move_no_network,
    opponent_choose_move=choose_move_randomly,
    game_speed_multiplier=3,
    render=False,
    verbose=True,
)


Resetting Game.
You are playing with the white tiles.
Black plays first


black places counter at coordinate: (4, 0)
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
3 3
white places counter at coordinate: (2, 2)
black places counter at coordinate: (1, 7)
[[ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0]
 [ 0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]


KeyboardInterrupt: Interrupted by user

In [98]:
gamma = 0.99

def minmax(
    env,
    action,
    V: Callable[[np.ndarray], int],
    max_depth=5,
    consecutive_pass=0,
    alpha=-float("inf"),
    beta=-float("inf"),
) -> Tuple[Optional[Tuple[int, int]], int]:
    """
    Computes the perfect minimax move with a max depth cap. When
    the cap is reached, V() is used to estimate the value.
    """
    

    new_env = copy.deepcopy(env)
    
    if action == None:
        observation, reward, done, info = new_env.reset()
    else:
        observation, reward, done, info = new_env.step(action)
    
    
    if consecutive_pass >= 2:
        return None, reward
    elif max_depth == 0:
        return None, V(state)
    legal_moves = info["legal_moves"]
    if len(moves) == 1:
        return (
            None,
            gamma * -minmax(env, 81, V, max_depth - 1, consecutive_pass + 1, beta, alpha)[1],
        )
    
    best_move, best_score = None, -float("inf")
    for action in legal_moves:
        score = (
            gamma * -minmax(new_env, action, V, max_depth - 1, 0, beta, alpha)[1]
        )
        if score >= best_score:
            best_move = action
            best_score = score
        if score >= -beta:
            break
        alpha = max(alpha, score)
    return best_move, best_score

In [101]:
env = GoEnv(
    opponent_choose_move,
    verbose=verbose,
    render=render,
    game_speed_multiplier=game_speed_multiplier,
)

env.reset()
env.observation.obs

Resetting Game.
You are playing with the black tiles.
Black plays first




AttributeError: 'numpy.ndarray' object has no attribute 'obs'